In [10]:
import imaplib
import smtplib
import email
import time
import ollama
import os
from email.header import decode_header
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from dotenv import load_dotenv

In [11]:
load_dotenv()

IMAP_SERVER = "imap.gmail.com"
SMTP_SERVER = "smtp.gmail.com"
EMAIL_ACCOUNT = os.getenv("s_email")
EMAIL_PASSWORD = os.getenv("s_password")
HUMAN_SUPPORT_EMAIL = os.getenv("prof_email")
check_model="llama3.1"
Omodel="deepseek-r1:1.5b"

In [12]:
def fetch_emails():
    try:
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)
        mail.select("inbox", readonly=False)

        status, email_ids = mail.search(None, "UNSEEN")
        email_ids = email_ids[0].split()

        if not email_ids:
            return []

        emails = []
        for email_id in email_ids:
            _, msg_data = mail.fetch(email_id, "(RFC822)")
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    # Decode subject
                    subject, encoding = decode_header(msg["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding if encoding else "utf-8")

                    sender = msg.get("From")  # Extract sender

                    # Extract email body
                    body = None
                    if msg.is_multipart():
                        for part in msg.walk():
                            content_type = part.get_content_type()
                            if content_type == "text/plain":
                                body = part.get_payload(decode=True).decode()
                                break
                    else:
                        body = msg.get_payload(decode=True).decode()

                    if body:
                        emails.append({"id": email_id, "sender": sender, "subject": subject, "body": body})

        return emails

    except Exception as e:
        print(f"❌ Error fetching emails: {e}")
        return []

In [13]:
def can_ai_handle(email_body):
    prompt = f"""
    You are an AI support assistant for KwikID. Your task is to classify customer emails into two categories:
    
    1. **AI**: Questions about KwikID features, account setup, login issues, basic troubleshooting.
    2. **HUMAN**: Refund requests, legal concerns, policy clarifications, complex technical issues.

    Email:
    {email_body}

    Respond with either 'AI' or 'HUMAN' only.
    """

    response = ollama.chat(model=check_model, messages=[{"role": "user", "content": prompt}])
    decision = response["message"]["content"].strip().upper()

    return decision


In [14]:
def generate_ai_reply(email_body):
    prompt = f"""
    You are KwikID's AI email assistant. Your task is to generate **a professional, helpful, and concise** response to customer emails.

    **Instructions:**
    - Answer as if you are a KwikID support agent.
    - Keep the response **direct and relevant** to the query.
    - If the sender's name is unclear, start with "Hi there".
    - Use a **friendly yet professional** tone.

    **Customer Email:**
    {email_body}

    **Your Response:**
    """

    response = ollama.chat(model=check_model, messages=[{"role": "user", "content": prompt}])
    return response["message"]["content"].strip()


In [15]:
def send_email(to_email, subject, body):
    try:
        server = smtplib.SMTP_SSL(SMTP_SERVER, 465)
        server.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)

        msg = MIMEMultipart()
        msg["From"] = EMAIL_ACCOUNT
        msg["To"] = to_email
        msg["Subject"] = f"Re: {subject}"

        msg.attach(MIMEText(body, "plain"))

        server.sendmail(EMAIL_ACCOUNT, to_email, msg.as_string())
        server.quit()
        print(f"✅ AI replied to: {to_email}")

    except Exception as e:
        print(f"❌ Error sending email: {e}")

In [16]:
def forward_to_human(original_sender, subject, body):
    try:
        server = smtplib.SMTP_SSL(SMTP_SERVER, 465)
        server.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)

        msg = MIMEMultipart()
        msg["From"] = EMAIL_ACCOUNT
        msg["To"] = HUMAN_SUPPORT_EMAIL
        msg["Subject"] = f"FWD: {subject}"

        forward_text = f"Forwarded from: {original_sender}\n\n{body}"
        msg.attach(MIMEText(forward_text, "plain"))

        server.sendmail(EMAIL_ACCOUNT, HUMAN_SUPPORT_EMAIL, msg.as_string())
        server.quit()
        print(f"📤 Forwarded complex email to human support.")

    except Exception as e:
        print(f"❌ Error forwarding email: {e}")

In [17]:
def process_emails():
    emails = fetch_emails()
    if not emails:
        print("🔄 No new emails.")
        return

    for email_data in emails:
        sender_email = email_data["sender"].split()[-1].strip("<>")  # Extract sender email
        subject = email_data["subject"]
        body = email_data["body"]

        print(f"\n📩 New Email from: {sender_email}\n📌 Subject: {subject}\n📖 Body: {body[:300]}...")

        # Check if AI can handle it
        if can_ai_handle(body):
            ai_reply = generate_ai_reply(body)
            send_email(sender_email, subject, ai_reply)
        else:
            forward_to_human(sender_email, subject, body)

In [18]:
if __name__ == "__main__":
    print("📡 Email AI Bot started! Press Ctrl+C to stop.")
    try:
        while True:
            process_emails()  # Process new emails
            print("\n🔄 Checking for new emails in 20 seconds...\n")
            time.sleep(20)
    except KeyboardInterrupt:
        print("\n🛑 Email AI Bot stopped.")

📡 Email AI Bot started! Press Ctrl+C to stop.

📩 New Email from: surya.pandidhar.btech2021@sitpune.edu.in
📌 Subject: Regarding login issues
📖 Body: My team is having trouble logging in to your software, can we please have a
check.

regards.

-- 
**This email is governed by the Disclaimer Terms of  SIU which may be 
viewed at **http://www.siu.edu.in/downloads/email-disclaimer.php 
<http://www.siu.edu.in/downloads/email-disclaimer.php>
...
✅ AI replied to: surya.pandidhar.btech2021@sitpune.edu.in

🔄 Checking for new emails in 20 seconds...


🛑 Email AI Bot stopped.
